# Import libraries

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import config
import json
import numpy as np
from shapely.geometry import LineString, mapping
from itertools import combinations
import re
from scipy import spatial

from load_preprocess.functions import Data
from load_preprocess.predictions import Number_Stations
from station_finder.functions import StationLocator, Scenarios


# Part 1: Determine the number of stations per region

In [2]:
config_path = 'params/config.json'
conf = json.load(open(config_path, "r"))

data = Data(path = 'data/')
df = data.create_df()

df.head(10)

100%|██████████| 10/10 [00:29<00:00,  2.93s/it]


,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line
3,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85932.399845,3076.692877,123934.499714,162432.991960
15,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147881.271737,4015.676440,126130.541927,208410.677869
10,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135138.054662,4994.602982,125508.904876,163440.964713
0,Alsace,0.000106,8.836421e+05,8.322648e+09,171172.687144,4374.466068,102940.396161,188809.381774
17,Picardie,0.000093,1.822331e+06,1.954720e+10,163238.986421,7854.875311,157760.172316,218702.689596
20,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203474.560283,4047.517513,239295.676544,319508.048270
13,Lorraine,0.000085,2.007468e+06,2.366142e+10,189457.934215,4562.427574,173570.336256,235280.099059
6,Bretagne,0.000085,2.333729e+06,2.760417e+10,244626.451176,4037.593151,187474.602114,285248.867037
4,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122829.920651,3688.358818,150571.399619,263736.945386
16,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228887.324618,6852.239326,202901.214557,276408.162313


## Scenario 1
### Meet all the demand in 2030 and all the demand in 2040

In [3]:
stations_num_1 = Number_Stations(df_data=df, path_conf='params/config.json', length_to_use='longest_line', scenario="scenario1")
df_f_1 = stations_num_1.final_station_calculation()
df_f_1.head(10)

,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line,perc_load,max_length_drive,...,R_2030_total,C_2030,h2_num_2040,R_2040_high_aut,R_2040_mid_aut,R_2040_low_aut,R_2040_total,C_2040,num_stations_2030,num_stations_2040
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,123.934500,162.432992,0.097211,720,...,210.603356,56.451613,4841.121182,259.498073,691.994862,97.311778,1048.804713,56.451613,4,19
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,126.130542,208.410678,0.066784,720,...,185.638899,56.451613,3325.859722,228.737744,609.967318,85.776654,924.481716,56.451613,3,16
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,125.508905,163.440965,0.039934,720,...,87.051273,56.451613,1988.699295,107.261527,286.030740,40.223073,433.515340,56.451613,2,8
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,102.940396,188.809382,0.029967,720,...,75.464104,56.451613,1492.353678,92.984224,247.957930,34.869084,375.811238,56.451613,1,7
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,157.760172,218.702690,0.040049,720,...,116.821359,56.451613,1994.449490,143.943184,383.848492,53.978694,581.770370,56.451613,2,10
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,239.295677,319.508048,0.113242,720,...,482.575254,56.451613,5639.468248,594.612313,1585.632834,222.979617,2403.224764,56.451613,9,43
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,173.570336,235.280099,0.030442,720,...,95.527101,56.451613,1515.989921,117.705145,313.880388,44.139430,475.724963,56.451613,2,8
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,187.474602,285.248867,0.066443,720,...,252.783700,56.451613,3308.872625,311.471215,830.589907,116.801706,1258.862828,56.451613,4,22
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,150.571400,263.736945,0.024451,720,...,86.008770,56.451613,1217.661778,105.976991,282.605310,39.741372,428.323673,56.451613,2,8
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,202.901215,276.408162,0.078611,720,...,289.808023,56.451613,3914.843916,357.091288,952.243435,133.909233,1443.243956,56.451613,5,26


In [4]:
print(f"For scenario 1: {df_f_1.num_stations_2030.sum()} stations are needed by 2030 and {df_f_1.num_stations_2040.sum()} stations are needed by 2040 \n")
stations_num_1.save_predictions(df_f_1)
stations_num_1.get_scenario_output(df_f_1)

For scenario 1: 61 stations are needed by 2030 and 307 stations are needed by 2040 

The output for scenario1 is 61 for 2030 and 307 for 2040.
Region Île-de-France: 4 stations for 2030 and 19 for 2040
Region Nord-Pas-de-Calais: 3 stations for 2030 and 16 for 2040
Region Haute-Normandie: 2 stations for 2030 and 8 for 2040
Region Alsace: 1 stations for 2030 and 7 for 2040
Region Picardie: 2 stations for 2030 and 10 for 2040
Region Rhône-Alpes: 9 stations for 2030 and 43 for 2040
Region Lorraine: 2 stations for 2030 and 8 for 2040
Region Bretagne: 4 stations for 2030 and 22 for 2040
Region Basse-Normandie: 2 stations for 2030 and 8 for 2040
Region Pays de la Loire: 5 stations for 2030 and 26 for 2040
Region Champagne-Ardenne: 2 stations for 2030 and 12 for 2040
Region Provence-Alpes-Côte d'Azur: 4 stations for 2030 and 21 for 2040
Region Poitou-Charentes: 2 stations for 2030 and 12 for 2040
Region Languedoc-Roussillon: 3 stations for 2030 and 13 for 2040
Region Franche-Comté: 1 stations f

## Scenario 2
### Meet 60% of the 2030 demand and 80% of the 2040 demand

In [5]:
stations_num_2 = Number_Stations(df_data=df, path_conf='params/config.json', length_to_use='longest_line', scenario="scenario2")
df_f_2 = stations_num_2.final_station_calculation()
df_f_2.head(10)

,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line,perc_load,max_length_drive,...,R_2030_total,C_2030,h2_num_2040,R_2040_high_aut,R_2040_mid_aut,R_2040_low_aut,R_2040_total,C_2040,num_stations_2030,num_stations_2040
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,123.934500,162.432992,0.097211,720,...,126.362014,56.451613,4666.140898,250.118625,666.983000,93.794484,1010.896109,56.451613,2,18
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,126.130542,208.410678,0.066784,720,...,111.383339,56.451613,3205.647925,220.470115,587.920306,82.676293,891.066714,56.451613,2,16
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,125.508905,163.440965,0.039934,720,...,52.230764,56.451613,1916.818598,103.384605,275.692279,38.769227,417.846111,56.451613,1,7
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,102.940396,188.809382,0.029967,720,...,45.278462,56.451613,1438.413183,89.623348,238.995595,33.608756,362.227699,56.451613,1,6
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,157.760172,218.702690,0.040049,720,...,70.092816,56.451613,1922.360954,138.740419,369.974450,52.027657,560.742525,56.451613,1,10
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,239.295677,319.508048,0.113242,720,...,289.545152,56.451613,5435.632046,573.120301,1528.320804,214.920113,2316.361218,56.451613,5,41
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,173.570336,235.280099,0.030442,720,...,57.316261,56.451613,1461.195105,113.450743,302.535314,42.544028,458.530085,56.451613,1,8
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,187.474602,285.248867,0.066443,720,...,151.670220,56.451613,3189.274819,300.213220,800.568585,112.579957,1213.361762,56.451613,3,21
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,150.571400,263.736945,0.024451,720,...,51.605262,56.451613,1173.649907,102.146498,272.390660,38.304937,412.842095,56.451613,1,7
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,202.901215,276.408162,0.078611,720,...,173.884814,56.451613,3773.343533,344.184374,917.824998,129.069140,1391.078512,56.451613,3,25


In [6]:
print(f"For scenario 2: {df_f_2.num_stations_2030.sum()} stations are needed by 2030 and {df_f_2.num_stations_2040.sum()} stations are needed by 2040 \n")
stations_num_2.save_predictions(df_f_2)
stations_num_2.get_scenario_output(df_f_2)

For scenario 2: 37 stations are needed by 2030 and 293 stations are needed by 2040 

The output for scenario2 is 37 for 2030 and 293 for 2040.
Region Île-de-France: 2 stations for 2030 and 18 for 2040
Region Nord-Pas-de-Calais: 2 stations for 2030 and 16 for 2040
Region Haute-Normandie: 1 stations for 2030 and 7 for 2040
Region Alsace: 1 stations for 2030 and 6 for 2040
Region Picardie: 1 stations for 2030 and 10 for 2040
Region Rhône-Alpes: 5 stations for 2030 and 41 for 2040
Region Lorraine: 1 stations for 2030 and 8 for 2040
Region Bretagne: 3 stations for 2030 and 21 for 2040
Region Basse-Normandie: 1 stations for 2030 and 7 for 2040
Region Pays de la Loire: 3 stations for 2030 and 25 for 2040
Region Champagne-Ardenne: 1 stations for 2030 and 12 for 2040
Region Provence-Alpes-Côte d'Azur: 3 stations for 2030 and 20 for 2040
Region Poitou-Charentes: 1 stations for 2030 and 11 for 2040
Region Languedoc-Roussillon: 2 stations for 2030 and 12 for 2040
Region Franche-Comté: 1 stations f

## Scenario 3
### Meet 150% of the 2030 demand and 90% of the 2040 demand

In [7]:
stations_num_3 = Number_Stations(df_data=df, path_conf='params/config.json', length_to_use='longest_line', scenario="scenario3")
df_f_3 = stations_num_3.final_station_calculation()
df_f_3.head(10)

,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line,perc_load,max_length_drive,...,R_2030_total,C_2030,h2_num_2040,R_2040_high_aut,R_2040_mid_aut,R_2040_low_aut,R_2040_total,C_2040,num_stations_2030,num_stations_2040
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,123.934500,162.432992,0.097211,720,...,315.905034,56.451613,4374.507092,234.486211,625.296562,87.932329,947.715102,56.451613,6,17
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,126.130542,208.410678,0.066784,720,...,278.458348,56.451613,3005.294929,206.690733,551.175287,77.509025,835.375044,56.451613,5,15
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,125.508905,163.440965,0.039934,720,...,130.576910,56.451613,1797.017436,96.923067,258.461512,36.346150,391.730729,56.451613,2,7
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,102.940396,188.809382,0.029967,720,...,113.196156,56.451613,1348.512359,84.021889,224.058371,31.508208,339.588468,56.451613,2,6
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,157.760172,218.702690,0.040049,720,...,175.232039,56.451613,1802.213394,130.069142,346.851047,48.775928,525.696117,56.451613,3,9
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,239.295677,319.508048,0.113242,720,...,723.862881,56.451613,5095.905043,537.300283,1432.800754,201.487606,2171.588642,56.451613,13,38
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,173.570336,235.280099,0.030442,720,...,143.290651,56.451613,1369.870411,106.360071,283.626856,39.885027,429.871954,56.451613,3,8
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,187.474602,285.248867,0.066443,720,...,379.175551,56.451613,2989.945143,281.449893,750.533049,105.543710,1137.526652,56.451613,7,20
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,150.571400,263.736945,0.024451,720,...,129.013155,56.451613,1100.296787,95.762342,255.366244,35.910878,387.039464,56.451613,2,7
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,202.901215,276.408162,0.078611,720,...,434.712035,56.451613,3537.509562,322.672851,860.460935,121.002319,1304.136105,56.451613,8,23


In [8]:
print(f"For scenario 3: {df_f_3.num_stations_2030.sum()} stations are needed by 2030 and {df_f_3.num_stations_2040.sum()} stations are needed by 2040 \n")
stations_num_3.save_predictions(df_f_3)
stations_num_3.get_scenario_output(df_f_3)

For scenario 3: 93 stations are needed by 2030 and 276 stations are needed by 2040 

The output for scenario3 is 93 for 2030 and 276 for 2040.
Region Île-de-France: 6 stations for 2030 and 17 for 2040
Region Nord-Pas-de-Calais: 5 stations for 2030 and 15 for 2040
Region Haute-Normandie: 2 stations for 2030 and 7 for 2040
Region Alsace: 2 stations for 2030 and 6 for 2040
Region Picardie: 3 stations for 2030 and 9 for 2040
Region Rhône-Alpes: 13 stations for 2030 and 38 for 2040
Region Lorraine: 3 stations for 2030 and 8 for 2040
Region Bretagne: 7 stations for 2030 and 20 for 2040
Region Basse-Normandie: 2 stations for 2030 and 7 for 2040
Region Pays de la Loire: 8 stations for 2030 and 23 for 2040
Region Champagne-Ardenne: 4 stations for 2030 and 11 for 2040
Region Provence-Alpes-Côte d'Azur: 6 stations for 2030 and 19 for 2040
Region Poitou-Charentes: 4 stations for 2030 and 11 for 2040
Region Languedoc-Roussillon: 4 stations for 2030 and 11 for 2040
Region Franche-Comté: 1 stations f

## Scenario 4
### Meet all the demand of 2030 and 2040 with the assumption that there will be 60% of high autonomy trucks, 35% of medium and 5% of low autonomy trucks 

In [9]:
stations_num_4 = Number_Stations(df_data=df, path_conf='params/config.json', length_to_use='longest_line', scenario="scenario4")
df_f_4 = stations_num_4.final_station_calculation()
df_f_4.head(10)

,region,road_density,length_m,area_m,length_max,length_mean,diameter,longest_line,perc_load,max_length_drive,...,R_2030_total,C_2030,h2_num_2040,R_2040_high_aut,R_2040_mid_aut,R_2040_low_aut,R_2040_total,C_2040,num_stations_2030,num_stations_2040
0,Île-de-France,0.000227,2.744410e+06,1.206353e+10,85.932400,3.076693,123.934500,162.432992,0.097211,720,...,245.078882,56.451613,4841.121182,471.814679,733.933945,14.744209,1220.492832,56.451613,4,22
1,Nord-Pas-de-Calais,0.000151,1.887368e+06,1.249483e+10,147.881272,4.015676,126.130542,208.410678,0.066784,720,...,216.027772,56.451613,3325.859722,415.886807,646.935034,12.996463,1075.818304,56.451613,4,19
2,Haute-Normandie,0.000113,1.398489e+06,1.237197e+10,135.138055,4.994603,125.508905,163.440965,0.039934,720,...,101.301466,56.451613,1988.699295,195.020959,303.365936,6.094405,504.481300,56.451613,2,9
3,Alsace,0.000106,8.836421e+05,8.322648e+09,171.172687,4.374466,102.940396,188.809382,0.029967,720,...,87.817491,56.451613,1492.353678,169.062225,262.985684,5.283195,437.331103,56.451613,2,8
4,Picardie,0.000093,1.822331e+06,1.954720e+10,163.238986,7.854875,157.760172,218.702690,0.040049,720,...,135.944881,56.451613,1994.449490,261.714881,407.112037,8.178590,677.005507,56.451613,2,12
5,Rhône-Alpes,0.000090,4.039422e+06,4.497380e+10,203.474560,4.047518,239.295677,319.508048,0.113242,720,...,561.572265,56.451613,5639.468248,1081.113296,1681.731794,33.784790,2796.629880,56.451613,10,50
6,Lorraine,0.000085,2.007468e+06,2.366142e+10,189.457934,4.562428,173.570336,235.280099,0.030442,720,...,111.164777,56.451613,1515.989921,214.009355,332.903442,6.687792,553.600589,56.451613,2,10
7,Bretagne,0.000085,2.333729e+06,2.760417e+10,244.626451,4.037593,187.474602,285.248867,0.066443,720,...,294.164100,56.451613,3308.872625,566.311300,880.928690,17.697228,1464.937218,56.451613,5,26
8,Basse-Normandie,0.000078,1.386823e+06,1.780635e+10,122.829921,3.688359,150.571400,263.736945,0.024451,720,...,100.088306,56.451613,1217.661778,192.685439,299.732905,6.021420,498.439763,56.451613,2,9
9,Pays de la Loire,0.000077,2.480511e+06,3.233398e+10,228.887325,6.852239,202.901215,276.408162,0.078611,720,...,337.249262,56.451613,3914.843916,649.256888,1009.955158,20.289278,1679.501324,56.451613,6,30


In [10]:
print(f"For scenario 4: {df_f_4.num_stations_2030.sum()} stations are needed by 2030 and {df_f_4.num_stations_2040.sum()} stations are needed by 2040 \n")
stations_num_4.save_predictions(df_f_4)
stations_num_4.get_scenario_output(df_f_4)

For scenario 4: 73 stations are needed by 2030 and 358 stations are needed by 2040 

The output for scenario4 is 73 for 2030 and 358 for 2040.
Region Île-de-France: 4 stations for 2030 and 22 for 2040
Region Nord-Pas-de-Calais: 4 stations for 2030 and 19 for 2040
Region Haute-Normandie: 2 stations for 2030 and 9 for 2040
Region Alsace: 2 stations for 2030 and 8 for 2040
Region Picardie: 2 stations for 2030 and 12 for 2040
Region Rhône-Alpes: 10 stations for 2030 and 50 for 2040
Region Lorraine: 2 stations for 2030 and 10 for 2040
Region Bretagne: 5 stations for 2030 and 26 for 2040
Region Basse-Normandie: 2 stations for 2030 and 9 for 2040
Region Pays de la Loire: 6 stations for 2030 and 30 for 2040
Region Champagne-Ardenne: 3 stations for 2030 and 14 for 2040
Region Provence-Alpes-Côte d'Azur: 5 stations for 2030 and 24 for 2040
Region Poitou-Charentes: 3 stations for 2030 and 14 for 2040
Region Languedoc-Roussillon: 3 stations for 2030 and 15 for 2040
Region Franche-Comté: 1 stations

# Part 2: Determine the location and size of the stations

## Load the data

In [3]:
shap_files = data.get_shapefiles()
csvs = data.get_csvs()
jsons = data.get_jsons()

100%|██████████| 9/9 [00:00<00:00, 128.59it/s]


In [4]:
station_locator = StationLocator(shap_files, csvs)
grid_size = conf["grid_size"]

In [9]:
station_locator = StationLocator(shap_files, csvs)
grid_size = conf["grid_size"]
sorted_locations = station_locator.get_best_location(grid_size=grid_size)

TypeError: StationLocator.__init__() got an unexpected keyword argument 'path_conf'

In [10]:
# update jsons inputs based on scenario and year you want to look at
scenarios = Scenarios(shap_files, csvs, jsons, path_conf="params/config.json")
top_locations = scenarios.distribute_locations(sorted_locations = sorted_locations,region_breakdown = jsons['output_scenario1']["num_stations_2030"])

In [11]:
polygons = scenarios.merge_closest_points(top_locations)
new_points = scenarios.fix_locations(polygons)
final_points = scenarios.get_size_station(new_points)

100%|██████████| 61/61 [01:13<00:00,  1.20s/it]


In [12]:
final_points_costs = scenarios.calculate_cost(final_points)

100%|██████████| 61/61 [00:00<00:00, 2259.36it/s]


## Compute the profitability of each station

In [13]:
# update jsons inputs based on scenario and year you want to look at
region_geom = station_locator.regions
regions_dem = jsons['output_refills_scenario1']["RT_2030_total"]

load_profit_by_station = scenarios.profitability_by_station(final_points, regions_dem)

0.3114754098360656 of stations are profitable
